In [2]:
import os
import math
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from PIL import Image

from skimage import io
from skimage.morphology import extrema
from skimage.measure import label

import skimage

import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters


import imageio

import javabridge 
import bioformats
javabridge.start_vm(class_path=bioformats.JARS, max_heap_size='8G')

#import net.imageJ.ImageJ




import sys
import time




#For the YOLO Network
import logging
import cv2
import torch
from torchvision import transforms as tf
import brambox.boxes as bbb
import lightnet as ln




In [3]:
# See your current version of python/anaconda
print (sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]


In [257]:
class ImagingParameters():
    def __init__(self, data_path, **kwargs):
        
        self.data_path = data_path
        for key, value in kwargs.items():
            setattr(self, key, value)
            #print (key, value, getattr(self, key))
            
            
        
        #Assert that the core variables are all set correctly.
        core_variables = ['channel_array', 'channel_thresholds', 'suffix', 'machine_learning_mode']
        print (core_variables)
        self.assert_variables(core_variables, "Core")

        
      
        
        #Store the dataframes from each of the experiments
        self.directory_list = self.find_files(self.data_path, {})
        self.dataframes = []
        #print(self.directory_list)
        
        
        #List containing all of the final dataframes for convenience
        self.experiments = []
        
        
        
                 
        machine_learning_args = [""]
        if (self.machine_learning_mode == True):
            
            network_variables = ['classes', 'network_size', 'labels', 'conf_thresh', 'nms_thresh', 'use_cuda']
            self.assert_variables(network_variables, "Network")              
                 
            self.initializeROINetwork()
            
            
        
        subframes = 0
        for items in self.channel_array:
            subframes += items
            
        self.frames_per_timepoint = subframes
        
        
        
        #If you are using the anisotropy analysis, create the relevant correction factors:

        
        if "Anisotropy" in self.image_type_array:
            
            
            #Assert that the correct variables are defined
            ani_variables = ['numerical_aperture', 'index_of_refraction', 'magnification', 'gFactor']
            self.assert_variables(ani_variables, "Anisotropy")
            
            
            
            asin = math.asin(self.numerical_aperture/self.index_of_refraction) 
            cos = math.cos(asin)

            kA = (2.0 - 3.0 * cos + cos * cos * cos) / (6.0 * (1.0 - cos))
            kB = (1.0 - 3.0 * cos + 3.0 * cos * cos - cos * cos * cos) / (24.0 * (1.0 - cos))
            kC = (5.0 - 3.0 * cos - cos * cos - cos * cos * cos) / (8.0 * (1.0 - cos))


            self.kA = kA
            self.kB = kB
            self.kC = kC

            print (kA, kB, kC)
            
            
            
            
            
            
            
            
            
    def assert_variables(self, variables, identifier = ""):
            for attribute in variables:
                assert hasattr(self, attribute), f"Error: {identifier} attribute {attribute} was not set.  Check your config file"


    def find_files(self, path, directory_list = {}):
        all_files = os.listdir(path)
        #print (all_files)
        for files in all_files:
            #print (path, os.path.dirname(path))

            #When you find the appropriate file, add the directory to the dictionary for processing
            if os.path.isfile(f'{path}{os.sep}{files}'):
                #print ("Found a file")
                if files.endswith(self.suffix):
                    #print (f"Ends with ome.tif ({files})")
                    prev_directory = os.path.dirname(path)
                    current_directory = path.replace(prev_directory, "")
                    if prev_directory in directory_list:
                        directory_list[prev_directory][current_directory] = None

                    else:
                        directory_list[prev_directory] = {current_directory : None}



            elif os.path.isdir(f'{path}{os.sep}{files}'):
                directory_list = self.find_files(f'{path}{os.sep}{files}', directory_list)
                #print ("Found a directory")


        return (directory_list)
        
        


    def iterate_through_files(self):
        """
        The purpose of this function is to iterate through all of the files in the directory list and then
        send them to the relevant analysis program
        """ 
        
        dataframes = self.dataframes
        suffix = self.suffix
        directory_list = self.directory_list
        debug_mode = self.debug_mode
        machine_learning_mode = self.machine_learning_mode


        if debug_mode == True:
            #Start a new dataframe each time
            dataframes = []



        #knowing the previous directory we looked at helps to determine whether we should append the
            #results to a current dataframe or not.  You want to append if the data are from the same cells
        prev_directory = ""
        new_dataframe = True


        for root_dir in directory_list:
            #note: there will usually only be one subdirectory per root_dir
            for directories in directory_list[root_dir]:
                print (os.path.dirname(os.path.dirname(directories)))
                if os.path.dirname(os.path.dirname(prev_directory)) == os.path.dirname(os.path.dirname(directories)):
                    print ("From the same core directory")
                    new_dataframe = False
                else:
                    print ("Not from the same core directory")
                    #if they do not share the same root directory, you need to create a new dataframe
                    new_dataframe = True
                    dataframes.append({})


                file_list = os.listdir(f'{root_dir}{directories}')


                for files in file_list:
                    if files.endswith(suffix):

                        #store all the relevant information about the image in a new class                    
                        current_Image = ImageClass(f'{root_dir}{directories}',
                                                  files, Parameters.channel_array,
                                                  Parameters.image_type_array, Parameters.channel_thresholds)

                        #Analyze the image
                        
                        dataframes = image_Analysis(current_Image=current_Image, 
                                                    Parameters = Parameters, 
                                                    dataframes = dataframes)



                prev_directory = directories


        #consolidate the dataframes
        dataframes = self.consolidate_dataframes()

        self.dataframes = dataframes
        return (dataframes)

            
            
    def consolidate_dataframes(self):

        """
        Combine the dataframes from each image into one consolidated dataframe
        
        """
        
        
        dataframes = self.dataframes
       
        for index, _  in enumerate(dataframes):   
            final_dataframe = pd.DataFrame([])
            for filenames in dataframes[index]:
                #print (f"current filename is: {filenames}")
                df_interest = dataframes[index][filenames]
                #print (type(df_interest))
                #print (df_interest)
                if len(final_dataframe)==0:
                    #print ("need a new dataframe")
                    final_dataframe = df_interest
                else:
                    #print ("Appending")
                    final_dataframe = final_dataframe.append(df_interest, ignore_index = True, sort = False)


            dataframes[index] = {"Total": final_dataframe}
            self.experiments = self.experiments.append(final_dataframe)
            #print (final_dataframe)


        self.dataframes = dataframes
        

        return (dataframes)
    
    
    
    def initializeROINetwork(self):
        #These will eventually be in a configuration file
        self.log = logging.getLogger('lightnet.detect')
        
        
        #self.classes = 20
        #NETWORK_SIZE = (416, 416)
        #self.network_size = [1482, 2535]  #Easier if a multiple of 13 or 26
        #self.labels = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle',
        #          'bus', 'car', 'cat', 'chair', 'cow',
        #          'diningtable', 'dog', 'horse', 'motorbike', 'person',
        #          'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
        #self.conf_thresh = .20
        #self.nms_thresh = .4
        #self.use_cuda = True
        

        #Use the GPU if available and wanted
        self.device = torch.device('cpu')
        if self.use_cuda:
            if torch.cuda.is_available():
                self.log.debug('CUDA enabled')
                self.device = torch.device('cuda')
            else:
                self.log.error('CUDA not available')
        
        
        
        self.network = ln.models.Yolo(self.classes,
                                      conf_thresh = self.conf_thresh,
                                      nms_thresh = self.nms_thresh,)
    
        self.network.postprocess.append(ln.data.transform.TensorToBrambox(self.network_size, self.labels))
        self.network.load(self.weights_path)
    
        self.network = self.network.to(self.device)
        
        
        
        
        
        
        
        
def create_pd_dataframe():
    
    dataframe = pd.DataFrame(columns=[(-1, -1,  "Image"), (-1,-1,"Index"), (-1, -1, "ROI")])
    #dataframe = pd.DataFrame(columns=["Image", "Index", "ROI"])
        
    
    return (dataframe)

#Should be a subclass of the imageing parameters
            
        
        

        

In [258]:
class ImageClass():
    def __init__(self, directory, filename, channel_array, image_type_array, channel_thresholds):
        self.directory = directory 
        self.filename = filename
        self.file_path = (f'{directory}{os.sep}{filename}')

        
        
        self.dir_name = directory.replace(os.path.dirname(directory), "")
        self.upper_dir_name = os.path.dirname(directory).replace (os.path.dirname(os.path.dirname(directory)), "")[1:]
        
        self.ROI_list = []
        
        
        image = Image.open(self.file_path)
        self.total_frames = image.n_frames
        self.width = image.width
        self.length = image.height
        
        self.channel_array = channel_array
        self.image_type_array = image_type_array
        
        self.channel_thresholds = channel_thresholds
        
        assert (len(channel_array) == len(image_type_array)), "Anisotropy and Channel arrays are not of equal length"
              
        self.channel_offset = np.zeros(len(channel_array))
        
        images_per_timepoint = 0
        for index, items in enumerate(channel_array):
            #print (index, items, images_per_timepoint)
            self.channel_offset[index] = images_per_timepoint
            images_per_timepoint += items
        
        self.images_per_timepoint = images_per_timepoint
        self.timepoints = self.total_frames // images_per_timepoint
        

In [259]:
class ROIs():
    def __init__(self, x, y, data, classification=None, confidence=None):
        """
            ______x_______
            |            |
           y|            |
            |            |
            |____________|
        
        
        Note: this standard was adopted as it conforms to both bramboxes and CV2 drawing formats.
        For numpy arrays, axis 0 is Y and axis 1 is X.  Therefore indexing should be
        np.array[y1:y1, x1:x2] when dealing with the images as numpy arrays
        
        """       
        
        self.x = x
        self.y = y
        self.data = data
        self.classification = classification
        self.confidence = confidence
        
        y_length, x_length = data.shape
        
        self.x_length = x_length
        self.y_length = y_length
        
    
#Should make this a subfunction of the ROI class



def calc_overlap(ROI1, ROI2, threshold = 0.8):
    """
    Calculate the IoU for two ROIs
    
    
    """
    x1, y1, x1Len, y1Len = ROI1.x, ROI1.y,  ROI1.x_length, ROI1.y_length
    x2, y2, x2Len, y2Len = ROI2.x, ROI2.y,  ROI2.x_length, ROI2.y_length
    
    
    box1_Area = (x1Len * y1Len)
    box2_Area = (x2Len * y2Len)
    #print ("b1 Area", box1_Area, "b2 Area", box2_Area)
    
    
    inter_Area = max(0, (min(x1+x1Len, x2+x2Len)-max(x1, x2)))*max(0, (min(y1+y1Len, y2+y2Len) - max(y1, y2)))
    
    
    
    if (box1_Area or box2_Area):
        IoU = inter_Area / (box1_Area + box2_Area - inter_Area + 0.001)
    else:
        IoU = 0
    
    
    
    
    return (IoU) 

In [260]:
def calculate_anisotropy(image, Parameters):
    """
    Parameters contains:
        NA - Numerical Aperture
        index_of_refraction - Index of Refraction for the immersion media
        mag - Magnification
        gFactor - G-Factor for the detector
        
        kA - Correctional Factors for high NA
        kB - Correctional Factors for high NA
        kC - Correctional Factors for high NA
        
        
    image.shape[-1] should be equal to 3 with channels: Open, Parallel, Perpendicular
    """
    
    #image.shape[-1] should be equal to 3 with channels: Open, Parallel, Perpendicular 
    assert image.shape[2] == 3, f"Anisotropy image was expecting 3 channels, but got: {image.shape[2]}"
    
    
    Ka = Parameters.kA
    Kb = Parameters.kB
    Kc = Parameters.kC
    
    
    G = Parameters.gFactor
    
    
    anisotropy_image = np.zeros((image.shape))

    Para = image[:,:,1]
    Perp = image[:,:,2]

    Ix = ((Kb * Para - Kc * Perp*G)/(Ka*Kb + Kb**2 -Ka*Kc - Kc**2))
    Iy = ((((Ka + Kb) * Perp*G) - (Ka+Kc)*Para)/(Ka*Kb+Kb**2-Ka*Kc-Kc**2))
    
    anisotropy_image[:,:,0] = Iy
    anisotropy_image[:,:,1] = Ix
    anisotropy_image[:,:,2] = 0
    
    numerator = np.add(Iy, np.multiply(Ix, -2*G))
    denominator = np.add(Iy, np.multiply(Ix, 2*G))
       
        
    anisotropy_image[:,:,-1] = np.divide (numerator, denominator, where=denominator!=0)
    #anisotropy_image[:,:,2] = np.divide((np.add(Iy, np.multiply(Ix, -2*G))),(np.add(Iy, np.multiply(Ix, 2*G)))) 
    
    return (anisotropy_image)

In [261]:
def image_Analysis(current_Image, Parameters, dataframes = []):
    """
    Analyze the current file.  Whether you are using machine learning or an algorithm, much of the process is the same
    
    current_Image:
        directory
        filename
        file_path
        timepoints
        total_frames
        width
        length
        channel_array
        channel_offset
        image_type_array
        images_per_timepoint
        
        dir_name
        upper_dir_name
        
    
    prev_images is a dictionary of all of the most current values from the previous images
        If a cell is found in the first image but not the second, it will still show up in the prev_images for the third image 
    
    
    dataframes is a list of all the dataframes associated with the analysis.
        different groups of data should have different dataframes (aka, data for a different graph)
        
        Dataframes is a list of dictionaries, each containing a dataframe for each image that it is analyzing.
        Once the analysis is complete, the dictionary will be replaced with a single entry dictionary
        {"Total": pd.DataFrame}
    
       
    """
        

        
        
    #g_factor = Parameters.g_factor    

        
    #Import the relevant parameters from the ImagingParameters Class   
    root_dir = Parameters.data_path
    suffix = Parameters.suffix
    debug_mode = Parameters.debug_mode
    machine_learning_mode = Parameters.machine_learning_mode
    
    #Import the relevant parameters from the ImageClass Class    
    directory = current_Image.directory
    total_frames = current_Image.total_frames
    timepoints = current_Image.timepoints
    filepath = current_Image.file_path
    filename = current_Image.filename
    channel_offset = current_Image.channel_offset
    image_type_array = current_Image.image_type_array
    images_per_timepoint = current_Image.images_per_timepoint
    upper_dir_name = current_Image.upper_dir_name
    channel_thresholds = current_Image.channel_thresholds
    
    
    #strings for the panda dataframe
    pre_string = ""

    
    general_strings = ['Image', 'ROI']
    anisotropy_strings = ['Para', 'Perp', 'AniPixel', 'AniAvg']
    intensity_strings = ['Intensity']
    
    
    para_intensity_str = f'Para'
    perp_intensity_str = f'Perp'
    anipix_intensity_str = f'AniPixel'
    aniavg_intensity_str = f'AniAvg'

    intensity_string = "Intensity"
    
    image_string = "Image"
    image_ROI_string = "ROI"
    


        
    if filename not in dataframes[-1]:
        new_dataframe = create_pd_dataframe()
        dataframes[-1].update({filename: new_dataframe})
    
    
    current_dataframe = dataframes[-1][filename]
    
    
    all_columns = current_dataframe.columns
    
    master_column_offset = 0
    if len(all_columns)>3:
        master_column_offset = max(all_columns[3:], key = lambda x: x[0])[0] + 1
    
    
    
    #print(f'root_dir: {Parameters.data_path}')
    #print (f'directory: {current_Image.directory}')
    #print (f'filepath: {current_Image.file_path}')
    #print (f'filename: {current_Image.filename}')
    
    
    
    
    
    ROI_match_threshold = 0.4
    
    
    print (f'current image is :{current_Image.filename} in {upper_dir_name} and it has {timepoints} timepoints and length df: {len(dataframes)}')
    for time in range(timepoints):
        
        #for string in general_strings:
        #    current_dataframe[(t, channel, string)] = np.nan
        
        for channel, (image_type, offset) in enumerate(zip(image_type_array, channel_offset)):
            #print (f"Current time is: {t}, anisotropy value is {anisotropy} and offset is {offset}")
            total_offset = time * images_per_timepoint + offset
            t = time + master_column_offset
            
            if image_type == "Anisotropy" or image_type == "Intensity":
            

                
                
                
                #Open the image
                
                Images = {}
                
                
                
                
                if image_type == "Anisotropy":
                    Images['Open'] = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=total_offset + 0, rescale=False))
                    Images['Para'] = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=total_offset + 1, rescale=False))
                    Images['Perp'] = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=total_offset + 2, rescale=False))

                    raw_image = np.dstack((Images['Open'], Images['Para'], Images['Perp']))
                    
                    anisotropy_image = calculate_anisotropy(raw_image, Parameters)
                    
                    Images['AniPixel'] = anisotropy_image[:,:,2].copy()
                                       
                    image_for_ROIs = Images['Open']
                    
                    for string in anisotropy_strings:
                            current_dataframe[(t, channel, string)] = np.nan
                    
                    
                    
                    
                elif image_type == "Intensity":
                    Images['Intensity'] = np.asarray(bioformats.load_image(filepath, c = 0, z=0, t=total_offset + 0, rescale=False))
                    image_for_ROIs = Images['Intensity']
                    
                    for string in intensity_strings:
                        current_dataframe[(t, channel, string)] = np.nan
                    
                    

                    
                #Preprocessing of the image
                
                
                thresholded_image = image_for_ROIs.copy()
                #if we change to floats, this should be np.nan, which doesn't exist for ints!
                thresholded_image[thresholded_image < channel_thresholds[channel]] = 0
                

                
                
                
                #Find ROIs and iterate through them
                
                if machine_learning_mode == True:
                    if image_type == "Anisotropy":
                        ROI_list = detect_ROIs(Parameters, raw_image)
                    elif image_type == "Intensity":
                        ROI_list = detect_ROIs(Parameters, np.dstack([image_for_ROIs, 
                                                                      image_for_ROIs*0.5, 
                                                                      image_for_ROIs*0.4]))
                
                else:
                    ROI_list = find_ROIs(image_for_ROIs) #for readability.  Can insert into the for loop  
                
                
                
                
                
                
                
                
                #Debug Mode: Output an image with all the ROIs
                if debug_mode == True:
                    debug_path = directory.replace(root_dir, f'{root_dir}{os.sep}debug')
                    if not os.path.isdir(debug_path):
                        os.makedirs(debug_path)
                        
                    debug_image = image_for_ROIs.copy()
                    debug_image = np.dstack([debug_image, debug_image*0.9, debug_image*0.8])
                    debug_image = ((debug_image/np.amax(debug_image))*255).astype('uint8')
                    
                
                #Output the ROIs as seperate images
                segmentation_outputs = False
                if segmentation_outputs == True:
                    segmentation_path = directory.replace(root_dir, f'{root_dir}{os.sep}segmentation')
                    if not os.path.isdir(segmentation_path):
                        os.makedirs(segmentation_path)

              
                
                

                for ROI in ROI_list:
                    match_index = 0
                    match_IoU = 0
                    for index, compROI in enumerate(current_dataframe[(-1, -1, "ROI")]):
                        #print ("Index is ", index)
                        #print (current_dataframe["ROI"])
                        current_dataframe.head(5)
                        IoU = calc_overlap(ROI, compROI)
                        if (IoU > max(ROI_match_threshold, match_IoU)):
                            match_index = index
                            match_IoU = IoU
                    
                    if match_IoU:
                        row_index = match_index #modify the current value
                        #change the ROI in the comparison row??
                        
                    else:
                        #append a new column
                        row_index = len(current_dataframe.index)
                        new_row = pd.DataFrame({(-1, -1, "ROI"): [ROI],
                                                (-1, -1, "Image"): filename,
                                                (-1, -1, "Index"): row_index})
                        current_dataframe = current_dataframe.append(new_row, ignore_index = True, sort=False)


                    #Use the ROI to append useful information to the dataframe    
                    x_corner = ROI.x
                    y_corner = ROI.y
                    x_len = ROI.x_length
                    y_len = ROI.y_length
                    
                    ROI_data = {}
                    
                    data_inputs = {}
                    #data_inputs[image_string] = current_Image
                    #data_inputs[image_ROI_string] = ROI
                    
                    
                    if debug_mode == True:
                        #print ("coordinates", x_corner, y_corner, x_len, y_len)
                        #print("stats:", (x_corner),(y_corner), (x_corner+x_len), (y_corner+y_len))
                              
                        
                        debug_image = cv2.rectangle(debug_image,
                                                    (x_corner,y_corner), 
                                                    (x_corner+x_len, y_corner+y_len), 
                                                    (255,0,0), 20)
                    
                    
                    
                    
                    if image_type == "Anisotropy":
                        
                        image_types = ['Para', 'Perp', 'AniPixel']
                        subRegion = {}
                        subRegionAvg = {}
                        
                        #subRegion['Thresh'] = thresholded_image[y_corner:y_corner+y_len, x_corner: x_corner+x_len].copy()
                        #subRegion[types] = np.multiply(subRegion[types], ROI.data)
                        
                        
                        
                        for types in image_types:
                            subRegion[types] = Images[types][y_corner:y_corner+y_len, x_corner: x_corner+x_len].copy()
                            subRegion[types] = np.multiply(subRegion[types], ROI.data)
                            data_inputs[types] = np.mean(subRegion[types][subRegion['Para']!=0])

                            
                                
                        para_value = data_inputs['Para']
                        perp_value = data_inputs['Perp']
                        data_inputs['AniAvg'] = (para_value - perp_value)/(para_value + 2*perp_value)
                        
                        #print (para_value, perp_value, data_inputs['AniAvg'])
                        
                        for types in data_inputs:
                            current_dataframe.at[row_index, (t, channel, types)] = data_inputs[types]


                      
                    
                    
                    elif image_type == "Intensity":
                        int_extract = Images['Intensity'][y_corner:y_corner+y_len, x_corner: x_corner+x_len].copy()
                        int_extract = np.multiply(int_extract, ROI.data)
                        int_value = np.mean(int_extract[int_extract!=0])
                        
                        current_dataframe.at[row_index, (t, channel, intensity_string)] = int_value
                        
                if debug_mode == True:
                    debug_save_path = f'{debug_path}{os.sep}{filename}'.replace(suffix, f'{channel}.jpg')
                    imageio.imwrite(debug_save_path, debug_image)
                        
                        
                
    
    #print (current_dataframe)
    dataframes[-1][filename] = current_dataframe            
    #print (dataframes[-1][filename].head(5))        
    return (dataframes)
            
            
            
    
    





#image_Analysis(Parameters = Parameters)


In [262]:
def find_local_maxima (data, neighborhood_size = 4, threshold = 700):
    """
    input:
    
    image = a 2D numpy matrix containing your data
    neighborhood_size = the area used to find the maximum filter (should be larger than your largest object)
    threshold = the minimum difference between your maximum and the lower value 
        in the neighbourhood (the background if the neighborhood size is large enough)
    
    
    
    output:
    The x and y coordinates of the maxima in the image (empty list if there are none...)
    """
    
    

    #find the maxima value within a specific neighborhood size    
    data_max = filters.maximum_filter(data, neighborhood_size)
    
    #set the maxima as "true" for the pooint where this maxima is true
    maxima = (data == data_max)
    
    #find the minima for comparison
    data_min = filters.minimum_filter(data, neighborhood_size)
    diff = ((data_max - data_min) > threshold)
    
    #Remove all points that are below the minimum threshold
    maxima[diff == 0] = 0


    
    #May not be necessary...
    labeled, num_objects = ndimage.label(maxima)
    slices = ndimage.find_objects(labeled)
    
    
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    
    #print (slices)
    x, y = [], []
    for dy,dx in slices:
        x_center = (dx.start + dx.stop - 1)//2
        x.append(int(x_center))
        y_center = (dy.start + dy.stop - 1)//2    
        y.append(int(y_center))

    #plt.imshow(data)
    #plt.savefig('/tmp/data.png', bbox_inches = 'tight')

    #plt.autoscale(False)
    #plt.plot(x,y, 'ro')
    #plt.savefig('/tmp/result.png', bbox_inches = 'tight')

    #print (x, y)
    return (x, y)

In [263]:
def find_ROIs (image, avg_radius = 4, thresh_tolerance = 0.5):
    test = 25

    seed = np.zeros(image.shape)
    x, y = find_local_maxima (image, 40, 800)

    overall_image = np.zeros(image.shape)


    ROI_List = []

    for (x_val, y_val) in (zip(x,y)):
        #Doesn't take care of the fact that y_val or x_val might be close to the edge (less than the tolerance)


        average_value = np.mean(image[y_val- avg_radius: y_val + avg_radius,
                                      x_val- avg_radius: x_val + avg_radius])





        if average_value:
            seed = np.zeros(image.shape)
            seed[y_val, x_val] = 1
            _ , mask = cv2.threshold(image,
                                     average_value * (1-thresh_tolerance),
                                     average_value*(1+thresh_tolerance),
                                     cv2.THRESH_BINARY )

            
            roi_image = ndimage.binary_propagation(seed, mask = mask)


            i, j = np.where(roi_image)
            x_corner = min(i)
            y_corner = min(j)
            roi_subimage = roi_image[min(i): max(i),
                                     min(j): max(j)]

            current_ROI = ROIs(x = min(j), y = min(i), confidence = None, classification = None, data = roi_subimage)

            
            #print (current_ROI)
            
            if not ROI_List:
                #print (current_ROI)
                ROI_List.append(current_ROI)
            
            else:           
                for compROI in ROI_List:
                    IoU_match = 0
                    IoU = calc_overlap(current_ROI, compROI)
                    if IoU>0.8:
                        IoU_match = 1
                        #print ("Already added a similar ROI")
                        break

                if not IoU_match:
                    #print (current_ROI)
                    ROI_List.append(current_ROI)

            
            
            
            
            #for testing purposes
            overall_image = np.add(overall_image, np.multiply(image, roi_image))

        #print (x_val, y_val, average_value, np.sum(roi_image), np.sum(mask), roi_subimage.shape)
        #print (x_corner, y_corner, min(j), max(j),min(i), max(i), roi_subimage.shape)

        

                
        
        


    #print (f"Numer of ROIS is {len(ROI_List)}")
    plt.imshow(overall_image)
    
    
    return (ROI_List)
    
    


In [264]:
def  create_summary_columns(df):
    
    dataframe = df.copy()
    
    #precolumns are the columns like ROI that aren't important for the export
    precolumns = 3
    
    summary_dataframe = pd.DataFrame([])
    
    timepoints = max(dataframe[precolumns:].columns, key = lambda x: x[0])[0]
    print (timepoints)
    
    
    channels = max(dataframe[precolumns:].columns, key = lambda x:x[1])[1]
    print (channels)
    
    
    
    statistics = {"Mean": np.mean, 
                  "Stdev": np.std,
                  "Count": len, "StdErr": lambda x: np.std(x)/np.sqrt(len(x))}
    
    
    
    for t in range (timepoints+1):
        
        series = pd.Series([t], index = [(" "," ","Time")])
        
        for func in statistics:
            new_series = dataframe.loc[0:, pd.IndexSlice[t,:,:]].apply(statistics[func])
            new_series.index = [(func, i[1], i[2]) for i in new_series.index]
            series = series.append(new_series)
        
        
        columns = series.index
        
        #print (series)
        summary_dataframe = summary_dataframe.append(series, ignore_index = True, sort = False)
        
    
    
    summary_dataframe.columns = pd.MultiIndex.from_tuples(summary_dataframe.columns, names = ['Stat', 'Channel', 'Para'])
    

    
    #print (df.columns)
    #print (summary_dataframe)
    return summary_dataframe


#create_summary_columns(sub_dataframe)

In [265]:
def create_cell_comparisons(df, Parameters):
    dataframe = df.copy()
    
    image_type_array = Parameters.image_type_array
    
    #image_type_bool = [image_type_array[i] == "Anisotropy" for i in range(len(image_type_array))]
    
    relative_values = False
    AniChannel = "AniAvg"
    IntChannel = "Intensity"
    
    #print (dataframe)
    
    statistics = {"Mean": np.mean, 
                  "Stdev": np.std,
                  "Count": len, "StdErr": lambda x: np.std(x)/np.sqrt(len(x))}
    
    
    cell_cell_comparisons = {}
    print ()
    
    for channel, image_type in enumerate(image_type_array):
        #print (channel, image_type)
        if image_type == "Anisotropy" or image_type == "Intensity":
            
            if image_type == "Anisotropy":
                channel_name = AniChannel
                
            elif image_type == "Intensity":
                channel_name = IntChannel
                
            
            
            channel_subarray = dataframe.loc[:, pd.IndexSlice[:, channel, channel_name]]
            channel_subarray = channel_subarray.dropna(how = 'any')
            
            
            
            #print (image_type, (0, channel, channel_name))
            first_channel = channel_subarray.loc[:, (0, channel, channel_name)].copy()
            
            if relative_values == True:        
                for columns in channel_subarray.columns:
                    channel_subarray.at[:, columns] = channel_subarray.loc[:, columns]/first_channel
                
            for functions in statistics:
                channel_subarray[functions] = channel_subarray.apply(statistics[functions], axis = 1)

            
            cell_cell_comparisons.update({f"{image_type}_Channel_{channel}": channel_subarray})
                
            
    
    
    
    #Need to find number of timepoints and then do a comparison for the rows where there is a value in every row
    
    #print (image_type_bool)
    
    
    #
    
    
    
    return (cell_cell_comparisons)



In [266]:
def parse_text_file(text_path):
    
    skip_characters = ["'", "#", "\n"]
    
    
    for char in ['/', '\\']:
        text_path = text_path.replace(char, os.sep)

    print (text_path)




    config_variables = {}
    with open(text_path, 'r') as f:
        for lines in f:
            if lines[0] not in skip_characters:           
                param_input = "".join(lines.split())
                variable, value = param_input.split('=')
                #print (variable, value, param_input)
                config_variables.update({variable: value})


    float_variables = ['numerical_aperture',
                       'index_of_refraction',
                       'magnification',
                       'gFactor',
                       'max_filter_region',
                       'max_threshold', 
                       'conf_thresh', 
                       'nms_thresh']
    
    int_variables = ['classes']
    
    binary_variables = ['debug_mode', 
                        'root_dir_same_treatment', 
                        'machine_learning_mode', 
                        'use_cuda']
    
    array_variables = {'channel_thresholds': "int",
                       'channel_array': 'int',
                       'image_type_array': 'str', 
                       'network_size':'int', 
                       'labels': 'str'}
    
    path_variables = ['weights_path']
    
    #Don't need this for anthing
    string_variables = ['suffix']
    
    
    for variables in config_variables:
    
    
        #Convert Floats
        if variables in float_variables:
            config_variables[variables] = float(config_variables[variables])
            
        elif variables in int_variables:
            config_variables[variables] = int(config_variables[variables])

        #Convert Binary Variables
        elif variables in binary_variables:
            config_variables[variables] = (config_variables[variables].lower() == "true")

        #Convert to arrays with the appropriate variable type
        elif variables in array_variables:
            var_type = array_variables[variables]
            for c in ['[', ']', "'", '"']:
                config_variables[variables] = config_variables[variables].replace(c,"")
            config_variables[variables] = config_variables[variables].split(',')
            if var_type.lower() == "int":
                config_variables[variables] = [int(x) for x in config_variables[variables]]
                
        elif variables in path_variables:
            for char in ['/', '\\']:
                config_variables[variables] = config_variables[variables].replace(char, os.sep)
                
        elif variables in string_variables:
            pass
        
        else:
            print (f'unknown variables found: {variables}')
    
    return config_variables

In [267]:
"""Detect Cells using Machine Learning"""


def post_transform(boxes, scale, pad):
    for box in boxes:
        box.x_top_left -= pad[0]
        box.y_top_left -= pad[1]

        box.x_top_left *= scale
        box.y_top_left *= scale
        box.width *= scale
        box.height *= scale
    return boxes

def output_ROIs (detections, im_h, im_w, segmentation = None):
    ROI_list = []
    for det in detections:

        #Note, the pixels are rounded down, but it shouldn't matter since the segmentation network will
        #do the pixel by pixel segmentation
        x = int(det.x_top_left)
        y = int(det.y_top_left)
        height = int(det.height)
        width = int(det.width)
        
        
        #Ensure that the ROIs do not go over the edge of the image.  Alternatively, don't use ROIs that go over the edge...
        if x<0:
            width = width+x
            x=0
        if y<0:
            height = height+y
            y=0
        height = min((im_h - y), height)
        width = min((im_w - x), width)
        
        print ("x", int(det.x_top_left),
               "y", int(det.y_top_left), 
               det.class_label, 
               det.confidence,
               "w", det.width,
               "h", det.height,
              "new_H", height, "new_W", width )
        
        
        fake_data = np.ones((height,width))
        
        
        roi = ROIs(x = x,
                   y = y, 
                   classification = det.class_label, 
                   confidence = det.confidence, 
                   data = fake_data)  #Use the data as ones for now... replace with segmentation later
        
        
        ROI_list.append(roi)
        
    return (ROI_list)



        
def detect_ROIs(Parameters, image):
    
    #Import the relevant parameters:
    network_size = Parameters.network_size
    network = Parameters.network
    net_w, net_h = Parameters.network_size
    device = Parameters.device
    log = Parameters.log

    
    
    
    
    save_check = False
    show_label = True
    use_cuda = True
    img = image.copy()
    
    #Maybe this should already be set.. 
    network.training = False
    
    im_h, im_w = image.shape[:2]

       
    device = torch.device('cpu')
    if use_cuda:
        if torch.cuda.is_available():
            log.debug('CUDA enabled')
            device = torch.device('cuda')
        else:
            log.error('CUDA not available')

            
    #print ("Image Shape:", img.shape)        
    

    #The control of the input should happen before this function...
    #If only one slice, pad to 3 using the anisotropy ratios
    #if img.shape[2] == 1:
    #    print ("There is only channel... extending")
    #    img = np.dstack([img, img*0.5, img*0.4])    
    
    
    #Prep the image for the neural network
    #print ("Pre conversion",img.shape)
    
    
    
    img = img/np.amax(img)
    img = img*255
    img = img.astype('uint8')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    #Transform the image and prep it for the 
    img_tf = ln.data.transform.Letterbox.apply(img, dimension = network_size)
    img_tf = tf.ToTensor()(img_tf)
    img_tf.unsqueeze_(0)
    img_tf = img_tf.to(device)


    # Run the image through the neural net
    with torch.no_grad():
        output = network(img_tf)
        
    if im_w == net_w and im_h == net_h:
        scale = 1
    elif im_w / net_w >= im_h / net_h:
        scale = im_w/net_w
    else:
        scale = im_h/net_h
        
    pad = int((net_w - im_w/scale) / 2), int((net_h - im_h/scale) / 2)

    
    #Convert the boxes into ROIs
    converted_boxes = []
    for b in output:
        converted_boxes.append(post_transform(b, scale, pad))
    
    output = converted_boxes
    
    
    image_markup = bbb.draw_boxes(img, output[0], color = (255,0,0), show_labels=show_label)
    if save_check:
        cv2.imwrite('detections.png', image)
    elif Parameters.debug_mode == True:
        cv2.imshow('image', image_markup)
        cv2.waitKey(1000)
        cv2.destroyAllWindows()
        print (output[0])
    
    
    
    
    
    #Add the segmentation algorithm here when it is ready...
    
    
    ROI_list = output_ROIs(output[0], im_h, im_w)
   
    #ROI_list = [ROIs(x=0, y=1000, data = np.ones((100,500)))]
        
    return ROI_list


In [268]:
"""

Start Here

"""

'\n\nStart Here\n\n'

In [271]:

#Work
data_path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline3"
text_path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline3{os.sep}config.txt"

#Home
data_path = f"C:{os.sep}Users{os.sep}William{os.sep}Documents{os.sep}Baseline"
text_path = f"C:{os.sep}Users{os.sep}William{os.sep}Dropbox{os.sep}Python Analysis{os.sep}config.txt"

config_variables = parse_text_file (text_path)

Parameters = ImagingParameters(data_path = data_path, **config_variables)

if Parameters.debug_mode == True:
    print("Loaded the following variables:")
    for variables in config_variables:
        print(f'{variables: <25}: {config_variables[variables]}')
        
    if (Parameters.machine_learning_mode == True):
        print ("Here is the network...")
        print (Parameters.network)



C:\Users\William\Dropbox\Python Analysis\config.txt
unknown variables found: same_cells
['channel_array', 'channel_thresholds', 'suffix', 'machine_learning_mode']


INFO       Loaded weights from C:\Users\William\Desktop\Lightnet\examples\yolo-voc\backup\final.pt


0.23898418155398782 0.014831490153302157 0.7461843282927101
Loaded the following variables:
channel_array            : [3, 3, 1]
image_type_array         : ['Anisotropy', 'Intensity', 'Brightfield']
channel_thresholds       : [500, 500, 500]
same_cells               : True
max_threshold            : 800.0
root_dir_same_treatment  : True
suffix                   : ome.tif
machine_learning_mode    : True
debug_mode               : True
numerical_aperture       : 1.4
index_of_refraction      : 1.53
magnification            : 63.0
gFactor                  : 1.0
max_filter_region        : 40.0
weights_path             : C:\Users\William\Desktop\Lightnet\examples\yolo-voc\backup\final.pt
classes                  : 20
network_size             : [1482, 2535]
labels                   : ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
conf_thresh     

In [241]:
start_time = time.time()
#work
#path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline3"

#home
#path = f"C:{os.sep}Users{os.sep}William{os.sep}Documents{os.sep}Baseline"

#analysis
#path = f"D:{os.sep}William{os.sep}2018-12-04 Test data from 08-3"


dataframes = []
dfs = Parameters.iterate_through_files()


for index, items in enumerate(dfs):
    save_path = f'{data_path}{os.sep}{index}.pkl'
    items['Total'].to_pickle(save_path)


end_time = time.time()


print (f"Total Time is {end_time - start_time}")


\
Not from the same core directory
current image is :Baseline_1_MMStack_1-Pos_000_003.ome.tif in Baseline and it has 2 timepoints and length df: 1
[Detection {class_label = cat, object_id = None, x = 1424.1996154373314, y = 398.4109812542173, w = 187.23547066754176, h = 146.70934959751392, confidence = 0.9631985425949097}, Detection {class_label = cat, object_id = None, x = 588.666003763917, y = 1102.834519441633, w = 228.42980808767715, h = 165.70254528956858, confidence = 0.9295817613601685}, Detection {class_label = cat, object_id = None, x = 767.3051369032557, y = 666.8211190536438, w = 146.93681255139805, h = 127.98357058794072, confidence = 0.883554995059967}, Detection {class_label = cat, object_id = None, x = 2018.5356649586708, y = 236.56511602774967, w = 206.0132806239984, h = 112.77046613397226, confidence = 0.8726407289505005}, Detection {class_label = cat, object_id = None, x = 1015.2972795947201, y = 1271.514528508772, w = 134.29010396898195, h = 130.236661553222, confide

[Detection {class_label = cat, object_id = None, x = 971.3632746605095, y = 1025.7558382886302, w = 199.51058354551495, h = 120.97196451822917, confidence = 0.9232482314109802}, Detection {class_label = cat, object_id = None, x = 1179.8139799995784, y = 1226.1556200447033, w = 191.714530913936, h = 138.32420721839154, confidence = 0.8162416815757751}, Detection {class_label = cat, object_id = None, x = 596.1157733953273, y = 1113.6371747427463, w = 172.67430174463354, h = 152.65655970605602, confidence = 0.7265763878822327}, Detection {class_label = cat, object_id = None, x = 252.4248462012905, y = 551.7756199392713, w = 197.3008039837424, h = 179.95230938580679, confidence = 0.68648362159729}, Detection {class_label = cat, object_id = None, x = 2261.063907515182, y = 602.0293416835358, w = 235.17574138358216, h = 148.31140301456014, confidence = 0.6821162700653076}, Detection {class_label = cat, object_id = None, x = 556.6730881252109, y = 231.9882667531208, w = 163.73274838425692, h 

x 244 y 567 cat 0.9463828206062317 w 180.08532797212382 h 150.26104085615933 new_H 150 new_W 180
x 677 y 806 cat 0.9431463479995728 w 182.20076728028005 h 115.82899563643936 new_H 115 new_W 182
x 525 y 120 cat 0.9223216772079468 w 132.76067003257845 h 117.71150600829749 new_H 117 new_W 132
x 843 y 328 cat 0.9010586738586426 w 170.32973365938133 h 166.55521251765984 new_H 166 new_W 170
x 152 y 1095 cat 0.8857040405273438 w 176.50961241934465 h 140.22155382823044 new_H 140 new_W 176
x 1154 y 1032 cat 0.8741595149040222 w 133.4026106243674 h 106.14875379554657 new_H 106 new_W 133
x 1230 y 1401 cat 0.790859580039978 w 229.73305446609314 h 85.32490298622534 new_H 79 new_W 229
x 1379 y 1381 cat 0.7824482917785645 w 340.0438935849992 h 123.65882023079033 new_H 99 new_W 340
x 800 y 629 cat 0.7625579833984375 w 142.89778991781589 h 115.8648108354947 new_H 115 new_W 142
x 2019 y 1246 cat 0.7139737010002136 w 234.61646726868253 h 190.1051895203378 new_H 190 new_W 234
x 2 y 518 cat 0.6503405570983

[Detection {class_label = cat, object_id = None, x = 1354.096464870108, y = 400.6454748650473, w = 216.36846717906548, h = 138.40749837898534, confidence = 0.9605494737625122}, Detection {class_label = cat, object_id = None, x = -5.943969561825236, y = 185.25661584851554, w = 237.71432581604253, h = 132.75240598773829, confidence = 0.9474706649780273}, Detection {class_label = cat, object_id = None, x = 1118.3477063301282, y = 363.335615932861, w = 155.09463513331858, h = 163.55053595621416, confidence = 0.9283941984176636}, Detection {class_label = cat, object_id = None, x = 786.9292631368085, y = 323.7179671685223, w = 175.4594800694269, h = 114.35592639011894, confidence = 0.9276727437973022}, Detection {class_label = cat, object_id = None, x = 710.0936880587889, y = 65.63395511766194, w = 136.40925233663336, h = 128.92309487943868, confidence = 0.8179194331169128}, Detection {class_label = cat, object_id = None, x = 955.8383228955804, y = 36.46913877994265, w = 134.03563645912408, 

[Detection {class_label = cat, object_id = None, x = 2078.5431980642716, y = 972.3726594129555, w = 174.276251047729, h = 123.19490148052675, confidence = 0.8865308165550232}, Detection {class_label = cat, object_id = None, x = 1047.6900448612519, y = 809.5588467864372, w = 191.5182240598115, h = 142.77295729087595, confidence = 0.8713799715042114}, Detection {class_label = cat, object_id = None, x = 1582.685307017544, y = 416.6638226425439, w = 130.16462771355222, h = 131.3266825605179, confidence = 0.8281444907188416}, Detection {class_label = cat, object_id = None, x = 995.4315048814947, y = 1270.302433894231, w = 225.38439694300357, h = 167.78953546068448, confidence = 0.6763175129890442}, Detection {class_label = cat, object_id = None, x = 527.311808762441, y = 565.2471322537112, w = 144.62038126133393, h = 133.41713452049595, confidence = 0.48840025067329407}, Detection {class_label = cat, object_id = None, x = 174.019310831541, y = 78.04548329959515, w = 105.29700407164621, h = 

...

[Detection {class_label = cat, object_id = None, x = 1424.055521729504, y = 398.5305040696694, w = 186.15603024049005, h = 149.20895636966515, confidence = 0.9454817175865173}, Detection {class_label = cat, object_id = None, x = 726.6383839142628, y = 653.8553923119097, w = 216.9232123371078, h = 159.51852800006327, confidence = 0.918810248374939}, Detection {class_label = cat, object_id = None, x = 988.7947610914306, y = 1273.9726588857964, w = 178.2682271898195, h = 139.7384260540549, confidence = 0.7566810250282288}, Detection {class_label = cat, object_id = None, x = 1613.0498284096661, y = 915.4202460779353, w = 161.99962851747006, h = 219.9188830418143, confidence = 0.6664296984672546}, Detection {class_label = cat, object_id = None, x = -18.377088374293606, y = 346.3052805541498, w = 183.37540690104169, h = 900.8150369538631, confidence = 0.5516310930252075}, Detection {class_label = cat, object_id = None, x = 518.4911707416077, y = 690.901426492915, w = 120.62107447574014, h = 

[Detection {class_label = cat, object_id = None, x = 244.05304342210695, y = 567.5992746288799, w = 180.08532797212382, h = 150.26104085615933, confidence = 0.9463828206062317}, Detection {class_label = cat, object_id = None, x = 677.1822791466346, y = 806.4341788967612, w = 182.20076728028005, h = 115.82899563643936, confidence = 0.9431463479995728}, Detection {class_label = cat, object_id = None, x = 525.2643038071441, y = 120.78904932101891, w = 132.76067003257845, h = 117.71150600829749, confidence = 0.9223216772079468}, Detection {class_label = cat, object_id = None, x = 843.2755771075828, y = 328.370819627193, w = 170.32973365938133, h = 166.55521251765984, confidence = 0.9010586738586426}, Detection {class_label = cat, object_id = None, x = 152.28954501544578, y = 1095.0828193530701, w = 176.50961241934465, h = 140.22155382823044, confidence = 0.8857040405273438}, Detection {class_label = cat, object_id = None, x = 1154.3805035425103, y = 1032.4772267206479, w = 133.402610624367

[Detection {class_label = cat, object_id = None, x = 1089.081483004386, y = 58.04560981781377, w = 240.75187635743507, h = 175.77731393255524, confidence = 0.9520514607429504}, Detection {class_label = cat, object_id = None, x = 1383.7221936677633, y = 958.0848937246964, w = 131.25291782641702, h = 83.52129797478598, confidence = 0.7120698094367981}, Detection {class_label = cat, object_id = None, x = 1225.1677868800607, y = 944.3262351341094, w = 144.94473526062754, h = 92.84909457105053, confidence = 0.6750535368919373}, Detection {class_label = cat, object_id = None, x = 1384.9366829031715, y = 204.131538092527, w = 119.01636615294998, h = 95.97525259164664, confidence = 0.6093252301216125}, Detection {class_label = cat, object_id = None, x = 1135.6647715291836, y = 364.35884784075574, w = 128.70569193604504, h = 98.58598799197136, confidence = 0.55391526222229}, Detection {class_label = cat, object_id = None, x = 543.9706135079285, y = -28.49276078567814, w = 628.9888683736083, h =

[Detection {class_label = cat, object_id = None, x = 1342.0271592442646, y = 393.4705660635965, w = 224.94914857192563, h = 139.3841653224106, confidence = 0.9632251858711243}, Detection {class_label = cat, object_id = None, x = -3.5479691849063766, y = 182.22742045167004, w = 233.8151542467949, h = 138.01290650567225, confidence = 0.958043098449707}, Detection {class_label = cat, object_id = None, x = 1125.8713968665654, y = 376.79838267543863, w = 146.54291043300861, h = 141.16296518508562, confidence = 0.9573594331741333}, Detection {class_label = cat, object_id = None, x = 722.180068372554, y = 75.28261138874832, w = 112.88253178770243, h = 122.92614433092949, confidence = 0.8730481863021851}, Detection {class_label = cat, object_id = None, x = 525.5637355294788, y = 249.56228518260798, w = 170.09446215919155, h = 177.42569805768179, confidence = 0.8589845895767212}, Detection {class_label = cat, object_id = None, x = 802.5506764771003, y = 334.9064687710864, w = 149.6711572397415,

[Detection {class_label = cat, object_id = None, x = 2042.3184147267207, y = 958.8717869644063, w = 216.85885690130527, h = 142.98472506668566, confidence = 0.9016116261482239}, Detection {class_label = cat, object_id = None, x = 1572.4131122954623, y = 395.121605094467, w = 163.32358055783993, h = 165.3698361457279, confidence = 0.849159300327301}, Detection {class_label = cat, object_id = None, x = 1071.0174002087551, y = 804.8678886217949, w = 154.01738109923247, h = 142.49097766953443, confidence = 0.7545015811920166}, Detection {class_label = cat, object_id = None, x = 1011.7928040127363, y = 1259.9295609817814, w = 195.50546301714323, h = 175.7953646887652, confidence = 0.6789085268974304}, Detection {class_label = cat, object_id = None, x = 540.6559897151232, y = 561.4771660973347, w = 136.2784201432819, h = 140.39741530527792, confidence = 0.6055877804756165}, Detection {class_label = cat, object_id = None, x = 174.183043033327, y = 65.18178822958839, w = 106.04076160013918, h 

In [ ]:
"""
Organize and output the dataframes to excel files


"""


sub_dataframe = dfs[0]["Total"]
columns = sub_dataframe.columns
sub_dataframe.loc[:, pd.IndexSlice[1,0,:]].mean()

#work
writer = pd.ExcelWriter("D:/Documents/Documents/0000 New Scope/Baseline3/test.xlsx", engine='xlsxwriter')

#home
#writer = pd.ExcelWriter("C:{os.sep}Users{os.sep}William{os.sep}Documents{os.sep}Baseline{os.sep}test.xlsx", engine='xlsxwriter')
#print (sub_dataframe)

sub_dataframe.to_excel(writer, "Raw_Data")
create_summary_columns(sub_dataframe).to_excel(writer, "Summary")

cell_cell_comparison = create_cell_comparisons(sub_dataframe, Parameters)
for key in cell_cell_comparison:
    cell_cell_comparison[key].to_excel(writer, key)

writer.save()



In [ ]:
"""


Cells beyond this point are not part of the main program...


"""

In [ ]:
class ImagingParameters():
    #Note: the input to __init__ may be replaced by just the config file path
    def __init__(self, data_path,
                 numerical_aperture=None, 
                 index_of_refraction=None, 
                 magnification = None, 
                 gFactor = None, 
                 channel_array = [1], 
                 image_type_array = ['Intensity'], 
                 channel_thresholds = 0, 
                 timepoints = 1, 
                 same_cells = False, 
                 max_filter_region = 5,
                 max_threshold =  600,
                 root_dir_same_treatment = True, 
                 suffix = "ome.tif", 
                 machine_learning_mode = False, 
                 debug_mode = False,
                 weights_path = "final.pt"):
        
        

        #    for key, value in **kwargs.iteritems():
        #        setattr(self, key, value)
        
        
        self.data_path = data_path
        self.numerical_aperture = numerical_aperture
        self.index_of_refraction = index_of_refraction
        self.magnification = magnification
        self.gFactor = gFactor
        self.channel_array = channel_array
        self.image_type_array = image_type_array
        self.max_filter_region = max_filter_region
        self.max_threhsold = max_threshold
        self.root_dir_same_treatment = root_dir_same_treatment
        self.suffix = suffix
        self.channel_thresholds = channel_thresholds
        self.machine_learning_mode = machine_learning_mode
        self.debug_mode = debug_mode
        
        
        self.weights_path = weights_path

In [ ]:

#create_cell_comparisons(sub_dataframe, Parameters)

In [ ]:
image_path = "D://Documents//Documents//0000 New Scope//Baseline3//Treat1//Baseline_1//Baseline_1_MMStack_1-Pos_000_003.ome.tif"
image = np.asarray(bioformats.load_image(image_path))
image = np.dstack([image, image*0.9, image*0.8])
print ("Max 1", np.amax(image))
image = ((image/np.amax(image))*255).astype('uint8')
print ("Max 2", np.amax(image))
#print (image)
image2 = cv2.rectangle(image, (200,0), (400, 1000), (255,0,0), 20)
plt.imshow(image2)

save_path = "D://Documents//Documents//0000 New Scope//Baseline3//debug//Treat1//Baseline_1//Baseline_1_MMStack_1-Pos_000_003.jpg"


imageio.imwrite(save_path, image2)

In [ ]:
#print (dfs[3])
for items in dfs:
    for keys in items:
        print (type(items[keys]), keys)
        #items[keys].pop("ROI")
        items[keys].to_csv("D:/Documents/Documents/0000 New Scope/Baseline3/test.csv")


"""final_dataframe = pd.DataFrame([])

for filenames in dfs[-1]:
    print (f"current filename is: {filenames}")
    df_interest = dfs[-1][filenames]
    print (type(df_interest))
    #print (df_interest)
    if len(final_dataframe)==0:
        print ("need a new dataframe")
        final_dataframe = df_interest
    else:
        print ("Appending")
        final_dataframe = final_dataframe.append(df_interest, ignore_index = True, sort = False)

print (final_dataframe)
"""

In [ ]:
array = np.ones((5,6))
array = np.pad(array, ((2,3), (0,1)), 'constant')
print (array)
print (array.shape)

In [ ]:
df = pd.DataFrame([], columns=["Image", "Index", "ROI"])
df.head()

imparameters = ImageParams(f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline{os.sep}Baseline_1",
                         "Baseline_1_MMStack_1-Pos_002_002.ome.tif", Parameters.channel_array, Parameters.image_type_array)

df2 = pd.DataFrame({"Image": [imparameters, 2], "ROI": [3,4]})
df3 = df.append(df2, sort=False)
df3.head()

print (df3.at[0, "Image"])

#for index, items in enumerate(df3["Image"]):
#    print (index, items)


In [ ]:
df2.dtypes

In [ ]:
np.array = []
array.append({"array": 1})
print ("array" in array[-1])
array.append([2,3])
print (array[-1][1])

In [ ]:
#Example full program::

In [ ]:
path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline_1"
dir_list = find_files(path)
iterate_through_files (dir_list)

In [ ]:
dff  = pd.DataFrame({("A", "B"): [1,2,3], ("A", "C"): [2,5,6], ("G"):[1,2,3]})
dff.at[2, ("A", "B")]= 100
for columns in dff.columns:
    print (columns[0])




In [ ]:
arr1 = np.array([1, 2, 3, 43, 6, 5])
arr2 = np.array([True, True, True, False, False, True])
arr4 = arr1[:4].copy()
arr3 = np.multiply (arr1, arr2)
print (arr3)
print (np.mean(arr3[arr3 !=0]))

print (arr4)
arr4[2] = 100
print (arr1)

In [ ]:
new_path = "D:/Documents/Documents/0000 New Scope/Baseline3/Treat1/Baseline_1/Baseline_1_MMStack_1-Pos_000_003.ome.tif"

import_image = np.asarray(bioformats.load_image(new_path, t=0, rescale = False)).astype('float')



In [ ]:
%%time
new_image = import_image.copy()

new_image [new_image <  10000] = np.nan

plt.imshow(new_image)
plt.colorbar()

In [ ]:
fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()


fgmask = fgbg.apply(import_image)
cv2.imshow('frame',fgmask)
k = cv2.waitKey(30) & 0xff


cv2.destroyAllWindows()

In [ ]:
"""
BF Matcher

"""


bio_loc = f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_002_002.ome.tif'

bio_para_import = np.asarray(bioformats.load_image(bio_loc, 
                                          c = 0, z=0, t=1, rescale=False))

bio_perp_import = np.asarray(bioformats.load_image(bio_loc, 
                                          c = 0, z=0, t=2, rescale=False))

bio_para = skimage.img_as_ubyte(bio_para_import)
bio_perp = skimage.img_as_ubyte(bio_perp_import)

filtered_para = cv2.bilateralFilter(bio_para, 9, 75, 75)

print (type(bio_para[0,0]))
print(cv2.imread(bio_loc, 0).shape)
#bio_para = cv2.imread(bio_loc, 0)

#bio_perp = cv2.imread(bio_loc, 0)

print (type(bio_para[0,0]))

b = cv2.normalize(bio_para_import, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype = cv2.CV_8U)



print (bio_para.shape)
print (bio_para[:4,:4], bio_perp[:4, :4])
print (b[:4,:4])
print(type(b[0,0]))


orb = cv2.ORB_create()

kp1, des1 = orb.detectAndCompute(bio_para, None)
kp2, des2 = orb.detectAndCompute(bio_perp, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)

matches = bf.match(des1, des2)
matches = sorted(matches, key = lambda x: x.distance)


img3 = cv2.drawMatches(bio_para, kp1, bio_perp, kp2, matches[:10], None, flags = 2)
cv2.imshow("Matches", img3)
#plt.imshow(bio_para)
#plt.show()


#fig, axs = plt.subpolots(1, 1)
#axs = plt.hist(bio_para)

#plt.show()


cv2.imshow("Para", bio_para)
cv2.imshow("Bilateral", filtered_para)
cv2.imshow("Perp", bio_perp)
cv2.imshow("Difference", (bio_para - bio_perp)*100)
cv2.imshow("Normalizes", b)
cv2.waitKey(0)
cv2.destroyAllWindows()
#print (bioformats.get_omexml_metadata(path=f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_000_001.ome.tif'))


print (kp1, des1)


In [ ]:
plt.plot([1,2,3,4], [5,6,7,8], 'ro')
plt.axis([0,10,0,20])
plt.show()


bio_para_reshape = bio_para_import.reshape(-1, 1)
print (bio_para_reshape.shape)
bio_para_reshape2 = filtered_para.reshape(-1, 1)

x = 100+ 15*np.random.randn(10000)
#n, bins, patches = plt.hist(bio_para_reshape, 50)
n, bins, patches = plt.hist(bio_para_reshape2, 50)
print (max(bio_para_reshape2))


In [ ]:
"""def find_maxima (image, sigma = 9, h_value = None):
    
    if not h_value:
        h_value = max(image)/1000
              
    
    proc_image = cv2.GaussianBlur(img, (sigma, sigma), 0)
    
    local_maxima = extrema.h_maxima(proc_image, h_value)
    
    
    
    
    
    return (local_maxima)
    
    
    
    

local_maximums = find_maxima(bio_para_import, 1, 1)

fig, ax = plt.subplots(1, 3, figsize= (15,5))

ax[0].imshow(bio_para_import)

ax[1].imshow(local_maximums)

print (find_maxima(bio_para_import, 1, 1))

x = np.arange(bio_para_import.shape[1])
y = np.arange(bio_para_import.shape[0])



#x, y = np.meshgrid(x, y)
#ax[2].scatter(x, y, c=local_maximums[x,y])
#ax[2].show()

plt.imshow(find_maxima(bio_para_import, 1, 50))
"""

In [ ]:
from skimage.feature import peak_local_max
xy = peak_local_max(bio_para_import, min_distance=5,threshold_abs=1200)
plt.scatter(xy[:, 0], xy[:, 1])

In [ ]:
x = np.arange(10)
y = np.arange(16)
test_data = np.random.randint(low=0,high=65535, size=(10, 16))

#scatter plot the measurements with
# x - measurement index (0-9 in this case)
# y - byte value index (0-15 in this case) 
# c = test_data[x,y]

x, y = np.meshgrid(x,y)
plt.scatter(x,y,c=test_data[x,y])
plt.show()

In [ ]:



data_path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline_1"

print (os.listdir(data_path)[4])
print (f'{data_path}{os.sep}{os.listdir(data_path)[4]}')

img = cv2.imread(f'{data_path}{os.sep}{os.listdir(data_path)[4]}')*100
#cv2.imshow("image", (img[:,:,1]*1000))
#cv2.waitKey(0)
#cv2.destroyAllWindows()

cv2.circle(img, (2000,1000), 400, (255,255,255), -1)
points = np.array([[1,1], [33,4], [100,20], [50, 400]], np.int32)
cv2.polylines(img, [points], True, (0,255,255), 3)
cv2.putText(img, "Hello, world!", (0,100), cv2.FONT_HERSHEY_COMPLEX, 4, (255, 0, 0))

plt.imshow(img)
plt.show()

#img = np.ones((100,100,3))*1
#img[:,:,2] = np.arange(0, 100*100, 1).resize((100,100))
print (img[:,:,2])

gg = calculate_anisotropy(image = img, Parameters = Parameters)


print (gg.shape)
cv2.imwrite(f'{path}{os.sep}{os.listdir(path)[7]}2.tiff', gg)

#print (sum(sum(img-gg)))

#cv2.imshow("image", (img-gg)*1000)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
"""Thresholding options
Works Well

from skimage.feature import peak_local_max
xy = peak_local_max(bio_para_import, min_distance=2,threshold_abs=1500)
plt.scatter(xy[:, 0], xy[:, 1])


Works Better:

import numpy as np
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
import matplotlib.pyplot as plt

fname = '/tmp/slice0000.png'
neighborhood_size = 40
threshold = 700


data = bio_para_import
#data = scipy.misc.imread(fname)

data_max = filters.maximum_filter(data, neighborhood_size)
maxima = (data == data_max)
data_min = filters.minimum_filter(data, neighborhood_size)
diff = ((data_max - data_min) > threshold)
maxima[diff == 0] = 0

labeled, num_objects = ndimage.label(maxima)
slices = ndimage.find_objects(labeled)
x, y = [], []
for dy,dx in slices:
    x_center = (dx.start + dx.stop - 1)/2
    x.append(x_center)
    y_center = (dy.start + dy.stop - 1)/2    
    y.append(y_center)

plt.imshow(data)
#plt.savefig('/tmp/data.png', bbox_inches = 'tight')

plt.autoscale(False)
plt.plot(x,y, 'ro')
#plt.savefig('/tmp/result.png', bbox_inches = 'tight')

print (x, y)


"""

In [ ]:
#find all of the files in the current path
print(os.path.isdir(path))



"""for items in os.listdir(path):
    #print (items, "file:",  os.path.isfile(f"{path}/{items}"), "dir:",  os.path.isdir(f'{path}/{items}'))
    img = cv2.imread(f'{path}/{items}')
    cv2.imshow('image', img*100)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
"""

    
    
    
"""
Options for going up ap directory:

1) split path by "/" __file__.rsplit(os.sep, 2)  or print(path.rsplit('/')[-3])
2) iterate up twice os.path.dirname(os.path.dirname(path))



"""

In [ ]:
"""
Testing the find_files function
"""

path2 = "D:\\Documents\\Documents\\0000 New Scope\\2018-08-01 Nuclear Parameter Test"
#path2 = "D:/Documents/Documents/0000 New Scope/2018-08-01 Nuclear Parameter Test/N2(600-40)/Baseline/Baseline_1"
dir_list = {}
result = find_files(path2, dir_list)


for items in result:
    print (f'{items}, {result[items]}')

In [ ]:
data_path = f"D:{os.sep}Documents{os.sep}Documents{os.sep}0000 New Scope{os.sep}Baseline_1"
print (data_path)
directory_list = find_files(data_path, {})
print (directory_list)
iterate_through_files(directory_list)


bio_im = np.asarray(bioformats.load_image(f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_000_001.ome.tif', 
                                          c = 0, z=0, t=6, rescale=False))


In [ ]:
import matplotlib.image as mpimg

img=mpimg.imread(f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_000_000.ome.tif')
print (img.shape)

im = io.imread(f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_000_000.ome.tif')
print ((im.shape))

pilim = Image.open(f'{data_path}{os.sep}Baseline_1_MMStack_1-Pos_000_000.ome.tif')
print (pilim.n_frames)
print (pilim)
print (pilim.info)
print (pilim.load())
pilim.width